## Integrantes do grupo  - T10 Paulista

> Amauri Silva **Matrícula:** A58330068

> Caio Ramos  **Matrícula:** A58338177

> Isabella Massaro **Matrícula:** A58337886

> Márcia Magalhães **Matrícula:** A58331043

---

# Objetivo do trabalho - Crianção de um Robô para operar Cripto Moedas

#### A proposta desse trabalho é criar um Robô baseado em um modelo de Rede Recorrente LSTM

Para um melhor aproveitamento/entendimento, alguns comentários foram adicionados no decorrer do código python ao invés de concentrar tudo em um bloco markdown


A proposta para utilizar uma rede LSTM foi mergulhar em uma alternativa de deeplearning com a finalidade de prover uma solução mais robusta na predição de preços de criptomoedas.
Consequentemente essa escolha trouxe um nível de complexidade maior ao projeto


---

# Modelo de deeplearning baseado em LSTM

## Excução do modelo - Pré requisito

Instalar as bibliotecas executando seguinte comando (a partir do diretório raiz src)

pip install -r requirements.txt


---

In [29]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from simple_robot import feature_eng
from IPython.display import display
from datetime import datetime
from sklearn.metrics import mean_squared_error
from simple_robot import feature_eng, api_get, api_post, get_result, compute_quantity, how_much_i_have
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import config
import os
import time
import math
import time
import numpy as np
import pandas as pd
import config

In [30]:
def dataset_process(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
    dataX.append(a)
    dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

### Obtendo os dados

O métod *stock_data()* é responsável por obter os dados, adicionalos em um objeto pandas e incrementar duas novas variáveis *forward_average* é *target*

In [31]:
def stock_data():
    print("[INFO] Obtendo os dados (dataset) ...")
    data_frame = pd.read_parquet(config.DATA_FRAME_PATH_DOWNLOAD)
    
    # Calculando qual a média de close dos próximos 10min
    data_frame['forward_average'] = data_frame[::-1]['close'].rolling(10).mean()[::-1].shift(-1)

    # Target será a diferença percentual do 'forward_average' com o 'close' atual
    data_frame['target'] = 100 * (data_frame['forward_average'] - data_frame['close']) / data_frame['close']

    return data_frame

### Processamento dos dados

Para que os dados possam ser ingeridos pela rede LSTM alguns tratamentos são feitos
Por exemplo o **MinMaxScaler**, transforma as features por escala, dimensiona e traduz cada feature individualmente de modo que esteja no intervalo determinado no conjunto de treinamento, por exemplo entre zero e um (*é feito o fit dos dados*)

A utilização do LSTM impleca em um tratamento fino dos dados, de certa forma foi trabalhoso conseguir equalizar os dados par que o processamento por parte da rede LSTM ocorresse



In [32]:
def pre_process_data():

    #
    data_frame = stock_data()
    data_frame = feature_eng(data_frame)

    #Remove linhas no qual contem valores NA (em uma pré analize foi contabilizada 39 linhas)
    data_frame = data_frame.dropna()

    # Mostra as variáveis do dataframe 
    display(data_frame)

    mms = MinMaxScaler(feature_range=(0,1))
    stock_data_transf = mms.fit_transform(np.array(data_frame).reshape(-1,1))
    training_size = round(len(stock_data_transf) * 0.80)

    #Faz o split dos dados em 80/20 (80% treinamento e 20% teste)
    train_data = stock_data_transf[:training_size]
    test_data = stock_data_transf[training_size:]

    time_step = 100
    X_train, y_train = dataset_process(train_data, time_step)
    X_test, y_test = dataset_process(test_data, time_step)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    return X_train, y_train, X_test, y_test, mms

### Treinamento da rede LSTM


Quanto a arquitetura da rede (setup), basicamente a rede LSTM comte 4 camadas com utilizando 1000 unidades/neuônios onde a camada de saida contem uma unidade apenas.

Foram feitos vários testes com diferentes quantidades de neurônios e camadas, de modo que esse setup atual mostrou-se melhor comparados com os testados anteriormente.

O função de otimização escolhida foi a Adam *(método stochastic gradient descent baseado em adaptive estimation )*

E a função Loss com  o padrão MSE

Apenas duas EPOCHs foram necessárias para reduzir o Loss

In [36]:
def traing_model(X_train, y_train, X_test, y_test, mms):
    print("[INFO] Treinando o modelo baseado em LSTM ...")
    model = Sequential()
    model.add(LSTM(1000, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(1000, return_sequences=True))
    model.add(LSTM(1000))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(X_train, X_test, epochs=2, batch_size=1, verbose=2)
    model.summary()

    print("[INFO] evaluating network...")
    predIdxs = model.predict(X_test, batch_size=1)

    print("[INFO] printing predIdxs ...")
    print(predIdxs)

    # Salvando o modelo
    model.save("lstm_model.model", save_format="wb")


In [37]:
if __name__ == '__main__':

    X_train, y_train, X_test, y_test, mms = pre_process_data()
    traing_model(X_train, y_train, X_test, y_test, mms)


[INFO] Obtendo os dados (dataset) ...


/home/amaurisilva/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,const,close,volume,number_of_trades,target,lag_1,lag_2,ma_10,ma_30,ratio_ma,time
open_time,,,,,,,,,,,
2019-07-05 12:29:00,1.0,0.003810,1.241574e+07,59,1.444208,0.761075,-0.763702,0.003874,0.003997,0.969227,0
2019-07-05 12:30:00,1.0,0.003810,4.488370e+06,68,1.678743,-0.010499,0.750656,0.003860,0.003984,0.968800,1
2019-07-05 12:31:00,1.0,0.003860,6.436490e+06,74,0.439895,1.295341,1.284978,0.003856,0.003969,0.971372,2
2019-07-05 12:32:00,1.0,0.003847,6.321878e+06,61,0.887680,-0.335317,0.964367,0.003845,0.003949,0.973784,3
2019-07-05 12:33:00,1.0,0.003840,1.257411e+06,18,1.221779,-0.177071,-0.512982,0.003837,0.003935,0.975138,4
...,...,...,...,...,...,...,...,...,...,...,...
2021-07-24 23:45:00,1.0,0.196970,5.038079e+05,199,0.134031,0.000000,0.045694,0.196645,0.196213,1.002200,1080676
2021-07-24 23:46:00,1.0,0.197040,1.036830e+06,209,0.125352,0.035529,0.035529,0.196710,0.196271,1.002235,1080677
2021-07-24 23:47:00,1.0,0.197150,2.287845e+06,263,0.063908,0.055795,0.091304,0.196791,0.196327,1.002365,1080678


[INFO] Treinando o modelo baseado em LSTM ...
Epoch 1/2
1/1 - 0s - loss: 5.5408e-07
Epoch 2/2
1/1 - 0s - loss: 0.0046
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 100, 1000)         4008000   
_________________________________________________________________
lstm_37 (LSTM)               (None, 100, 1000)         8004000   
_________________________________________________________________
lstm_38 (LSTM)               (None, 1000)              8004000   
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 1001      
Total params: 20,017,001
Trainable params: 20,017,001
Non-trainable params: 0
_________________________________________________________________
[INFO] evaluating network...
[INFO] printing predIdxs ...
[[0.00479404]]
INFO:tensorflow:Assets written to: lstm_model.mo

---

# CryptoBOT

Método responsável por carregar o modelo gerado anteriromente

In [27]:
def load_lstm_model():
    print("[INFO] Carregando modelo criado no notebook anterior ...")
    model = load_model(config.LSTM_MODEL_PATH)
    return model

---
#### Método responsável por executa a as transações baseadas nas predições do modelo LSTM gerado

In [ ]:
def crypto_bot(token, ticker, valor_compra_venda):
    print("[INFO] Loop que checa o preço a cada minuto e toma as decisões de compra e venda ...")
    model = load_lstm_model()
    while True:
        # Pegando o OHLC dos últimos 500 minutos
        df = api_post('cripto_quotation', {'token': token, 'ticker': ticker})

        # Realizando a engenharia de features
        df = feature_eng(df)
        df_last = df.iloc[-1]

        # Calculando tendência baseada no modelo LSTM
        mms = MinMaxScaler(feature_range=(0, 1))
        test = np.expand_dims(df_last, axis=0)
        stock_data_transf = mms.fit_transform(np.array(test).reshape(-1, 1))
        tendencia = model.predict(stock_data_transf)

        # A quantidade de cripto que será comprada/ vendida depende do valor_compra_venda e da cotação atual
        qtdade = compute_quantity(coin_value=df_last['close'], invest_value=valor_compra_venda, significant_digits=2)

        # Print do datetime atual
        print('-------------------')
        now = pd.to_datetime('now')
        print(f'{now}')

        if tendencia > 0.02:
            # Modelo detectou uma tendência positiva
            print(f"Tendência positiva de {str(tendencia)}")

            # Verifica quanto dinheiro tem em caixa
            qtdade_money = how_much_i_have('money', token)

            if qtdade_money > 0:
                # Se tem dinheiro, tenta comprar o equivalente a qtdade ou o máximo que o dinheiro permitir
                max_qtdade = compute_quantity(coin_value=df_last['close'], invest_value=qtdade_money, significant_digits=2)
                qtdade = min(qtdade, max_qtdade)

                # Realizando a compra
                print(f'Comprando {str(qtdade)} {ticker}')
                api_post('buy', payload={'token': token, 'ticker': ticker, 'quantity': qtdade})

        elif tendencia < -0.02:
            # Modelo detectou uma tendência negativa
            print(f"Tendência negativa de {str(tendencia)}")

            # Verifica quanto tem da moeda em caixa
            qtdade_coin = how_much_i_have(ticker, token)

            if qtdade_coin > 0:
                # Se tenho a moeda, vou vender!
                qtdade = min(qtdade_coin, qtdade)
                print(f'Vendendo {str(qtdade)} {ticker}')
                api_post('sell', payload={'token': token, 'ticker': ticker, 'quantity': qtdade})
        else:
            # Não faz nenhuma ação, espera próximo loop
            print(f"Tendência neutra de {str(tendencia)}. Nenhuma ação realizada")

        # Print do status após cada iteração
        print(api_post('status', payload={'token': token}))
        time.sleep(60)


---
#### Método principal no qual executa a chamada dos métodos responsávies por fazer as transações

In [ ]:
if __name__ == "__main__":
    token = 'ba0b83cb57fcddf'
    ticker = 'DOGEUSDT'
    valor_compra_venda = 10

    crypto_bot(token, ticker, valor_compra_venda)